# Introduction to SQLite for AI and Data Science

SQLite is a lightweight, serverless, self-contained SQL database engine. 
It is often used for local applications, prototyping, or situations where a full RDBMS (like Microsoft SQL Server or MySQL) is overkill.

## SQLite vs Microsoft SQL Server (RDBMS)
| Feature                     | SQLite                        | Microsoft SQL Server         |
|-----------------------------|-------------------------------|------------------------------|
| **Type**                   | Serverless                    | Requires a server            |
| **Setup Complexity**       | Minimal setup                 | Complex installation         |
| **Portability**            | Single file (.db)             | Needs server and storage     |
| **Concurrency**            | Limited concurrent writes     | Supports high concurrency    |
| **Use Case**               | Small to medium applications  | Enterprise applications      |

SQLite is ideal for AI and data science projects where data size is manageable, and simplicity is preferred.


## Conda Environment Setup

Before starting, we will set up a virtual environment using Conda and install the necessary libraries.

### Steps:
1. Open the terminal or Anaconda prompt.
2. Run the following commands:

```bash
conda create -n sqlite_demo python=3.9 ipykernel
conda activate sqlite_demo
conda install sqlite
pip install pandas
```

## Start working with sqlite

Choose the environment you just created as a kernel for this notebook and follow the next steps.

In [2]:
# Import Required Libraries
import sqlite3
import pandas as pd

In [5]:
# Connect to SQLite database (creates a file called 'student_data.db')
conn = sqlite3.connect("student_data.db")
cursor = conn.cursor()
print("Database connection established successfully.")

Database connection established successfully.


### Create Tables with Primary and Foreign Keys

In [6]:
# Drop tables if they already exist (for clean re-runs)
cursor.execute("DROP TABLE IF EXISTS Students")
cursor.execute("DROP TABLE IF EXISTS Grades")

In [7]:
# Create Students table
cursor.execute("""
    CREATE TABLE Students (
        StudentID INTEGER PRIMARY KEY,
        FirstName TEXT NOT NULL,
        LastName TEXT NOT NULL,
        Age INTEGER NOT NULL
    );
""")

# Create Grades table with a foreign key referencing Students
cursor.execute("""
    CREATE TABLE Grades (
        GradeID INTEGER PRIMARY KEY,
        StudentID INTEGER,
        Subject TEXT NOT NULL,
        Grade TEXT NOT NULL,
        FOREIGN KEY (StudentID) REFERENCES Students(StudentID)
    );
""")
print("Tables created successfully.")

Tables created successfully.


### Insert Data into Tables

In [8]:
# Insert data into Students table
cursor.execute("INSERT INTO Students (FirstName, LastName, Age) VALUES ('Alice', 'Smith', 20);")
cursor.execute("INSERT INTO Students (FirstName, LastName, Age) VALUES ('Bob', 'Johnson', 22);")

# Insert data into Grades table
cursor.execute("INSERT INTO Grades (StudentID, Subject, Grade) VALUES (1, 'Math', 'A');")
cursor.execute("INSERT INTO Grades (StudentID, Subject, Grade) VALUES (1, 'Science', 'B');")
cursor.execute("INSERT INTO Grades (StudentID, Subject, Grade) VALUES (2, 'Math', 'C');")

# Commit the changes
conn.commit()
print("Data inserted successfully into Students and Grades tables.")

Data inserted successfully into Students and Grades tables.


### Perform SELECT Query with JOIN

In [9]:
# Fetch all students and their grades using JOIN
query = """
    SELECT 
        Students.FirstName, 
        Students.LastName, 
        Grades.Subject, 
        Grades.Grade
    FROM Students
    INNER JOIN Grades ON Students.StudentID = Grades.StudentID;
"""

# Execute the query and load results into a DataFrame
df = pd.read_sql_query(query, conn)
print("Results of the JOIN query:")
print(df)


Results of the JOIN query:
  FirstName LastName  Subject Grade
0     Alice    Smith     Math     A
1     Alice    Smith  Science     B
2       Bob  Johnson     Math     C


### Create a Procedure-like Function in Python

In [10]:
# Since SQLite doesn't support stored procedures, we use Python functions instead

def fetch_student_grades(student_id):
    """
    Fetch grades for a specific student ID.
    """
    query = """
        SELECT 
            Students.FirstName, 
            Students.LastName, 
            Grades.Subject, 
            Grades.Grade
        FROM Students
        INNER JOIN Grades ON Students.StudentID = Grades.StudentID
        WHERE Students.StudentID = ?;
    """
    result = cursor.execute(query, (student_id,))
    rows = result.fetchall()
    print(f"\nGrades for Student ID {student_id}:")
    for row in rows:
        print(row)

# Call the function to fetch grades for StudentID = 1
fetch_student_grades(1)



Grades for Student ID 1:
('Alice', 'Smith', 'Math', 'A')
('Alice', 'Smith', 'Science', 'B')


### Close the Database Connection

In [11]:
conn.close()
print("Database connection closed.")

Database connection closed.
